In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-09-12 00:44:02--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-12 00:44:02 (13.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [18]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]
        
        

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields = ['course']
)
    

In [20]:
index.fit(documents)

In [23]:
q = 'The course has already started. Can I still enroll'

boost = {'question': 3.0, 'course': 0.5}

res = index.search(
    query = q,
    boost_dict = boost,
    filter_dict = {'course': 'data-engineering-zoomcamp'},
    num_results = 5
)

res

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [24]:
import openai

In [29]:
from openai import OpenAI
import os

In [32]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key = api_key)


In [34]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{'role': 'user', 'content': q}]
)
    

In [35]:
response.choices[0].message.content

'If the course has already started, whether or not you can still enroll typically depends on the policies of the institution or platform offering the course. Here are a few steps you can take:\n\n1. **Check the Course Page:** Review the course description or FAQ section. Sometimes, there is information about late enrollment.\n  \n2. **Contact the Instructor or Administration:** Reach out to the course instructor or the administration office to inquire about the possibility of late enrollment. They may allow it under certain conditions.\n\n3. **Automatic Enrollment Cut-off:** Some online platforms have an automatic cut-off for enrollment, but they sometimes allow joining a course late if you catch up quickly.\n\n4. **Catch-up Resources:** Ask if there are resources or recorded lectures that can help you catch up with the material already covered.\n\n5. **Look for Similar Courses:** If you cannot join this particular course, check if there are similar courses or sessions that will start 

In [36]:
prompt_template = f"""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {query}
    
    CONTEXT: 
    {search_results}
    """

context = ""
for doc in res:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [46]:
prompt = prompt_template.format(query = q, search_results = context).strip()


In [47]:
new_response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{'role': 'user', 'content': prompt}]
)
new_response

ChatCompletion(id='chatcmpl-A6SndwzIe8OCf2LhUkizn3UvWWWd1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Yes, even if the course has already started, you can still enroll. You will be eligible to submit the homeworks. However, keep in mind that there will be deadlines for turning in the final projects, so it is advisable not to delay everything until the last minute.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726104037, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=55, prompt_tokens=509, total_tokens=564))

In [62]:
def search(query):
    boost = {'question': 3.0, 'course': 0.5}

    res = index.search(
        query = query,
        boost_dict = boost,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        num_results = 10
    )
    return res
    

In [63]:

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [64]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [65]:
query = 'the course already started. can i enroll?'
results = search(query)
prompt = build_prompt(query, results)
llm(prompt)

"Yes, you can still join the course after the start date. Even if you don't register, you're eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's important not to leave everything for the last minute."